In [5]:
from unsloth import FastLanguageModel
import torch

# Önceki eğitimden checkpoint yolu
previous_checkpoint = "C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim\\checkpoint-1000"  # XXXX son checkpoint numarası

# Model ve tokenizer'ı yükle (4-bit quantize edilmiş halde)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=previous_checkpoint,
    max_seq_length=2048,
    dtype=torch.bfloat16,
    load_in_4bit=True,
)

FileNotFoundError: C:\Users\Mehmet\Desktop\Denizhan\model_egitim\checkpoint-1000/*.json (invalid repository id)

In [2]:
# Modeli LoRA ile yapılandırma
model = FastLanguageModel.get_peft_model(  # Modeli LoRA ile optimize edilmiş bir PEFT (Parameter-Efficient Fine-Tuning) modeline dönüştürür
    model,                                 # LoRA'nın uygulanacağı ana model (önceki adımda yüklenen model)
    r=32,                                  # LoRA'nın rank değeri: 16’dan 32’ye artırıldı (RTX A5000’in 24 GB VRAM’i daha yüksek rank için yeterli, daha iyi performans için) eskisi ise 16
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # LoRA'nın uygulanacağı Transformer katmanları (dikkat mekanizması ve feed-forward katmanları, değişmedi)
    lora_alpha=32,                         # LoRA'nın ölçeklendirme faktörü: 16’dan 32’ye artırıldı (r ile uyumlu, daha güçlü öğrenme için) eskisi ise 16
    lora_dropout=0.05,                     # LoRA için dropout oranı: 0’dan 0.05’e artırıldı (büyük veri setinde overfitting riskine karşı) eskisi ise 0
    bias="none",                           # Bias ayarı: "none" (değişmedi, optimize edilmiş bir seçenek)
    use_gradient_checkpointing="unsloth",  # Gradient checkpointing ayarı: "unsloth" (değişmedi, VRAM’i %30 azaltır, 2 kat büyük batch size sağlar)
    random_state=3407,                     # Rastgele durum tohumu: 3407 (değişmedi, tekrarlanabilirlik için sabit)
    use_rslora=True,                       # Rank Stabilized LoRA (RSLoRA) kullanımı: False’tan True’ya değiştirildi (stabiliteyi artırır, büyük veri setlerinde faydalı) eskisi ise False
    loftq_config=None,                     # LoftQ (Low-Rank Factorized Quantization) yapılandırması: None (değişmedi, ihtiyaç yoksa ek optimizasyon gerekmez)
)    

Unsloth: Already have LoRA adapters! We shall skip this step.


In [3]:
from datasets import load_from_disk

# Kaydedilmiş tokenize veri setini yükle
CHECKPOINT_DIR =  "C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim" #Model verilerini kayıt edeceğiniz klasörü seçin.
CHECKPOINT_DIR3 =  "C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim3" #Model verilerini kayıt edeceğiniz klasörü seçin.

tokenized_dataset = load_from_disk(f"{CHECKPOINT_DIR}/final_model/tokenized_dataset")

# Eğitim için dataset'i ayarla
dataset = tokenized_dataset

In [4]:
def add_attention_mask(example):
    example["attention_mask"] = [1] * len(example["input_ids"])
    return example

dataset = dataset.map(add_attention_mask, num_proc=4)

In [5]:
# Eğitim öncesi optimizasyon
torch.cuda.empty_cache()
model.gradient_checkpointing_enable()
torch.backends.cuda.matmul.allow_tf32 = True

In [6]:
# Gerekli kütüphaneleri içe aktarma
from trl import SFTTrainer              # TRL kütüphanesinden SFTTrainer’ı içe aktarır, denetimli ince ayar için kullanılır
from transformers import TrainingArguments  # Hugging Face Transformers’tan eğitim argümanlarını içe aktarır
from unsloth import is_bfloat16_supported   # Unsloth’tan bfloat16 desteğini kontrol eden bir fonksiyon alır

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,  # Burası değişti
    # Diğer parametreler aynı kalacak
    dataset_text_field=None,
    max_seq_length=2048,
    dataset_num_proc=2,
  packing=False,
        args=TrainingArguments(             # Eğitim argümanlarını tanımlar
        per_device_train_batch_size=16,  # Her cihaz (GPU) için eğitim batch boyutu: 8 (24 GB VRAM ile mümkün, performans artırıldı)
        gradient_accumulation_steps=4,  # Gradyan biriktirme adımları: 4 (efektif batch size’ı 8*4=32 yapar, bellek yönetimi için)
        warmup_steps=100,               # Öğrenme oranı ısınma adımları: 100 (büyük veri seti için daha yavaş ve stabil ısınma, 5’ten artırıldı)
        num_train_epochs=2,           # Tam bir epoch için eğitim (yorum satırı, şu anda kullanılmıyor, veri seti büyük olduğu için adım bazlı tercih ediliyor)
        max_steps=1000,                 # Toplam eğitim adımı sayısı: 1000 (60’tan artırıldı, 5 milyon satır için yeterli öğrenme sağlamak için)
        learning_rate=2e-4,             # Öğrenme oranı: 0.0002 (modelin ne kadar hızlı öğreneceğini belirler, büyük veri setinde uygun)
        fp16=not is_bfloat16_supported(),  # Float16 kullanımı: bfloat16 desteklenmiyorsa True (RTX A5000’de bfloat16 destekleniyor)
        bf16=is_bfloat16_supported(),   # Bfloat16 kullanımı: destekleniyorsa True (Ampere mimarisi için RTX A5000’de etkin, daha verimli)
        logging_steps=10,               # Log kayıt sıklığı: her 10 adımda bir (1’den artırıldı, büyük veri setinde log sıklığını azaltır)
        optim="adamw_8bit",             # Optimize edici: 8-bit AdamW (bellek verimli bir versiyon, değişmedi)
        weight_decay=0.01,              # Ağırlık çürümesi: 0.01 (overfitting’i önlemek için regularization, değişmedi)
        lr_scheduler_type="linear",     # Öğrenme oranı zamanlayıcısı: doğrusal (lineer bir şekilde azalır, değişmedi)
        seed=3407,                      # Rastgele tohum: 3407 (tekrarlanabilirlik için, değişmedi)
        output_dir=CHECKPOINT_DIR3,      # Çıktıların kaydedileceği dizin: "CHECKPOINT_DIR" (değişmedi)
        #report_to="tensorboard",               # Eğitim raporlama: "none" (WandB gibi araçlar kullanılmayacak, değişmedi)
        
    tf32=True,  # A5000 için TF32 aktivasyonu
    dataloader_pin_memory=True,  # Veri yükleme hızı için
    logging_dir=f"{CHECKPOINT_DIR3}/logs",

    #dataloader_num_workers=4,                # Veri yükleme iş parçacığı sayısı
    ), 
)

Applying chat template to train dataset (num_proc=2): 100%|██████████| 2324213/2324213 [22:38<00:00, 1710.84 examples/s]


In [7]:
try:                                    # Hata yakalama bloğu başlatır, checkpoint ile devam etmeyi dener
    print("Checkpoint aranıyor...")     # Kullanıcıya checkpoint’in arandığını bildirir
    trainer.train(resume_from_checkpoint=True)  # Eğitime son checkpoipnt’ten devam etmeyi dener (önceki bir eğitim varsa)
except Exception as e:                  # Eğer bir hata oluşursa (örneğin, checkpoint bulunamazsa) bu blok çalışır
    print(f"Hata: {str(e)[:200]}...\nYeni eğitim başlatılıyor...")  # Hatanın ilk 200 karakterini yazdırır ve yeni eğitime geçeceğini bildirir
    trainer.train() 

Checkpoint aranıyor...
Hata: No valid checkpoint found in output directory (C:\Users\Mehmet\Desktop\Denizhan\model_egitim3)...
Yeni eğitim başlatılıyor...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,324,213 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
10,1.561500
20,1.565000
30,1.515700
40,1.520400
50,1.491900
60,1.508000
70,1.518800
80,1.479900
90,1.481200
100,1.444400


KeyboardInterrupt: 

In [8]:
# ----------------------------------------------
# 10. Modeli Kaydet
# ----------------------------------------------
model.save_pretrained(f"{CHECKPOINT_DIR3}/final_model3")  # Eğitilmiş modeli belirtilen dizine kaydeder (CHECKPOINT_DIR/final_model)
tokenizer.save_pretrained(f"{CHECKPOINT_DIR3}/final_model3")  # Tokenizer’ı aynı dizine kaydeder

# ----------------------------------------------
# 11. Opsiyonel: Eğitim İlerlemesini İzleme ve Log Kaydetme
# ----------------------------------------------
trainer.save_model(f"{CHECKPOINT_DIR3}/final_checkpoint3")  # Eğitim sırasında ara checkpoint’leri kaydet (opsiyonel, büyük veri seti için faydalı)
print(f"Eğitim tamamlandı. Model şu adreste kaydedildi: {CHECKPOINT_DIR3}/final_model3")

Eğitim tamamlandı. Model şu adreste kaydedildi: C:\Users\Mehmet\Desktop\Denizhan\model_egitim3/final_model3


In [ ]:
# Token’lanmış veri setini kaydet
#trainer.train_dataset.save_to_disk(f"{CHECKPOINT_DIR}/final_model/tokenized_dataset")

Bu alan Bir Test Alanıdır.

In [10]:
import json

checkpoint_path = "C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim3\\checkpoint-500\\adapter_config.json"

with open(checkpoint_path, "r", encoding="utf-8") as f:
    config = json.load(f)

print(config.get("base_model_name_or_path", "Base model path not found"))


C:\Users\Mehmet\Desktop\Denizhan\Meta-Llama-3.1-8B-bnb-4bit


In [1]:
from unsloth import FastLanguageModel
from peft import PeftModel

# Step 1: Load the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim\\merged_model",  # Path to the base model
    load_in_4bit=True,
)

# Step 2: Load the LoRA adapters using PEFT
model = PeftModel.from_pretrained(
    model,
    "C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim3\\checkpoint-500" # Path to the LoRA checkpoint
)

# Step 3: Merge LoRA into the base model and unload
model = model.merge_and_unload()



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\Mehmet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.988 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


C:\Users\Mehmet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\unsloth\models\llama.py:1277: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.17s/it]
C:\Users\Mehmet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\peft\tuners\lora\bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [2]:
# Alpaca formatı için prompt şablonu tanımlama
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""  # Üç bölümden oluşan bir şablon: talimat (instruction), giriş (input) ve cevap (response) yer tutucuları içerir

In [2]:
# alpaca_prompt = Yukarıdan kopyalandığını varsayıyorum
# Alpaca formatında prompt şablonunun "instruction", "input", "response" alanlarını içerdiğini kabul ediyorum

# Modeli tahmin moduna geçirme
FastLanguageModel.for_inference(model)  # Modeli inference için hazırlar, Unsloth’un 2 kat hızlı yerel tahmin özelliğini etkinleştirir

# Girişi hazırlama ve tokenize etme
inputs = tokenizer(                     # Tokenizer ile giriş metnini token’lara çevirir
    [                                   # Tek bir prompt’u liste içinde işler
        alpaca_prompt.format(           # Alpaca şablonunu belirli değerlerle doldurur
            "Ankara hakkında bilgi ver.",  # Talimat: Maç sonuçlarını listele ve açıklama ekle
            "",                         # Giriş: Boş bırakıldı, model kendi bilgisine dayanacak
            "",                         # Çıkış: Boş bırakılır, modelin üretmesi için
        )
    ],
    return_tensors="pt"                 # Çıktıyı PyTorch tensör formatında döndürür
).to("cuda")                            # Tensörleri CUDA (GPU) belleğine taşır

# Model ile tahmin yapma
outputs = model.generate(               # Modeli kullanarak metin üretir
    **inputs,                           # Tokenize edilmiş girişleri modele verir
    max_new_tokens=1024,                 # Üretilecek maksimum yeni token sayısı: Daha fazla ayrıntı için 512
    use_cache=True                      # Önbellek kullanımını etkinleştirir, hızı artırır
)

# Çıktıyı decode etme ve yazdırma
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  # Üretilen token’ları insan tarafından okunabilir metne çevirir, özel token’ları atlar
print(result)

NameError: name 'FastLanguageModel' is not defined

Test 2

In [3]:
from unsloth import FastLanguageModel
from peft import PeftModel

# Step 1: Load the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim3\\merged_model3",  # Path to the base model
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


C:\Users\Mehmet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


FileNotFoundError: C:\Users\Mehmet\Desktop\Denizhan\model_egitim3\merged_model3/*.json (invalid repository id)

In [14]:
# alpaca_prompt = Yukarıdan kopyalandığını varsayıyorum
# Alpaca formatında prompt şablonunun "instruction", "input", "response" alanlarını içerdiğini kabul ediyorum

# Modeli tahmin moduna geçirme
FastLanguageModel.for_inference(model)  # Modeli inference için hazırlar, Unsloth’un 2 kat hızlı yerel tahmin özelliğini etkinleştirir

# Girişi hazırlama ve tokenize etme
inputs = tokenizer(                     # Tokenizer ile giriş metnini token’lara çevirir
    [                                   # Tek bir prompt’u liste içinde işler
        alpaca_prompt.format(           # Alpaca şablonunu belirli değerlerle doldurur
            "Fenerbahçe'nin son maçlarının sonuçlarını listele ve her maç için kısa bir açıklama yap.",  # Talimat: Maç sonuçlarını listele ve açıklama ekle
            "",                         # Giriş: Boş bırakıldı, model kendi bilgisine dayanacak
            "",                         # Çıkış: Boş bırakılır, modelin üretmesi için
        )
    ],
    return_tensors="pt"                 # Çıktıyı PyTorch tensör formatında döndürür
).to("cuda")                            # Tensörleri CUDA (GPU) belleğine taşır

# Model ile tahmin yapma
outputs = model.generate(               # Modeli kullanarak metin üretir
    **inputs,                           # Tokenize edilmiş girişleri modele verir
    max_new_tokens=512,                 # Üretilecek maksimum yeni token sayısı: Daha fazla ayrıntı için 512
    use_cache=True                      # Önbellek kullanımını etkinleştirir, hızı artırır
)

# Çıktıyı decode etme ve yazdırma
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  # Üretilen token’ları insan tarafından okunabilir metne çevirir, özel token’ları atlar
print(result)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Fenerbahçe'nin son maçlarının sonuçlarını listele ve her maç için kısa bir açıklama yap.

### Input:


### Response:
Fenerbahçe, 2019-20 sezonunda 3. Lig'de mücadele etti ve ligde 2. sırada bitirdi. Ligde 3. sıradaki Kocaeli Belediyespor'u 2 puan farkla geçerek, 4. sıradaki Gaziosmanpaşa'yı 4 puan farkla geçerek, 3. lig şampiyonu oldu ve 2. Lig'e yükseldi.


In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel

# Step 1: Load the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="C:\\Users\\Mehmet\\Desktop\\Denizhan\\Meta-Llama-3.1-8B-bnb-4bit",  # Path to the base model
    load_in_4bit=True,
)

# Step 2: Load the LoRA adapters using PEFT
model = PeftModel.from_pretrained(
    model,
    "C:\\Users\\Mehmet\\Desktop\\Denizhan\\model_egitim3\\checkpoint-500"  # Path to the LoRA checkpoint
)

# Step 3: Merge LoRA into the base model and unload
model = model.merge_and_unload()



==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.988 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [15]:
# alpaca_prompt = Yukarıdan kopyalandığını varsayıyorum
# Alpaca formatında prompt şablonunun "instruction", "input", "response" alanlarını içerdiğini kabul ediyorum

# Modeli tahmin moduna geçirme
FastLanguageModel.for_inference(model)  # Modeli inference için hazırlar, Unsloth’un 2 kat hızlı yerel tahmin özelliğini etkinleştirir

# Girişi hazırlama ve tokenize etme
inputs = tokenizer(                     # Tokenizer ile giriş metnini token’lara çevirir
    [                                   # Tek bir prompt’u liste içinde işler
        alpaca_prompt.format(           # Alpaca şablonunu belirli değerlerle doldurur
            "Fenerbahçe'nin son maçlarının sonuçlarını listele ve her maç için kısa bir açıklama yap.",  # Talimat: Maç sonuçlarını listele ve açıklama ekle
            "",                         # Giriş: Boş bırakıldı, model kendi bilgisine dayanacak
            "",                         # Çıkış: Boş bırakılır, modelin üretmesi için
        )
    ],
    return_tensors="pt"                 # Çıktıyı PyTorch tensör formatında döndürür
).to("cuda")                            # Tensörleri CUDA (GPU) belleğine taşır

# Model ile tahmin yapma
outputs = model.generate(               # Modeli kullanarak metin üretir
    **inputs,                           # Tokenize edilmiş girişleri modele verir
    max_new_tokens=512,                 # Üretilecek maksimum yeni token sayısı: Daha fazla ayrıntı için 512
    use_cache=True                      # Önbellek kullanımını etkinleştirir, hızı artırır
)

# Çıktıyı decode etme ve yazdırma
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  # Üretilen token’ları insan tarafından okunabilir metne çevirir, özel token’ları atlar
print(result)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Fenerbahçe'nin son maçlarının sonuçlarını listele ve her maç için kısa bir açıklama yap.

### Input:


### Response:

